In [ ]:
# -*- coding: utf-8 -*-

#copy and paste this into Petfinder_Prediction_Notebook

from shutil import copyfile
from os.path import join, abspath
from os import listdir, getcwd


import pandas as pd

from sklearn.ensemble import RandomForestClassifier

#directory where additional modules have to be fed to Kaggle
src = join("/kaggle", "input") 
#directory where modules can be imported in the Kaggle kernel
dst = join(getcwd())

#take a dict
modules_to_migrate = {
        "FeatureEngineer" : ["RescueGroupsFeature", "SentimentFeature",
                            "FeatureEngineerPipeline", "FeatureEngineerHelpers"],
        "ExperimentHelpers" : ["ExperimentData", "Evaluator", "Experiment"],
        "Study" : ["Study"]
        }
for k,v in modules_to_migrate.items():
    
    #copy our file into the working directory (make sure it has .py suffix)
    filename = str(k + ".py")
    copyfile(src = join(src, "petfinder", filename), dst = join(dst, filename))
    
    #I didn't want to do the import * because I wanted to make clear what was
    #getting imported.
    for className in v:
        exec(str("from " + k + " import " + className))

study = Study()

study.fe_pipeline.kwargs = {
    "feature_removal_kwargs" : {
        "correlation_threshold" : 0.10,
        "degree" : 1,
        "cv" : 3,
        "max_iter" : 1,
        "n_jobs" : -1,
        "verbose" : 2,
        # "selected_cols" : ["PhotoAmt", "Age", "RescuerGroup", "Breed1", "State", "MaturitySize",
        # "SentencesAmtGroup", "SentimentScoreGroup", "PureBred"]
        #"selected_cols" : ["PhotoAmt", "Age", "RescuerGroup", "Breed1", "SentencesAmtGroup", "SentimentScoreGroup", "PureBred"],
        "f_classif_threshold" : 0.1,
        "mutual_info_threshold" : 0.001
    }
}

train = pd.read_csv(join(src, "petfinder-adoption-prediction", "train", "train.csv"))
test = pd.read_csv(join(src, "petfinder-adoption-prediction", "test", "test.csv"))

exp = Experiment()
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
parameters = {
    "learning_rate" : [0.1, 0.01],
    "max_depth" : [2, 3, 4],
    "max_features" : ["auto", "sqrt", None],
    "subsample" : [1.0, 0.9, 0.8]
}
# rfc = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
gbdt = GradientBoostingClassifier(n_estimators=100, loss="deviance", random_state=0)
exp_model = GridSearchCV(gbdt, parameters, cv=5)
#from xgboost import XGBClassifier
study.produce_kaggle_submission(train, exp_model, test)
